# P8FS Getting Started - Build an Agent

Let's build a weather agent with P8FS MemoryProxy.

**Prerequisites:** Start PostgreSQL first:
```bash
cd p8fs && docker-compose up postgres -d
```

In [1]:
# To use the cluster tidb
  # from p8fs folder `docker compose down tidb`
  # and you can connect to the cluster with first forwarding a port in a new terminal
  # kc port-forward svc/fresh-cluster-tidb -n tikv-cluster 4000:4000
# then set the provider
# P8FS_STORAGE_PROVIDER=tidb
import os
#os.environ['P8FS_STORAGE_PROVIDER'] = 'tidb'
#os.environ['P8FS_TIDB_DATABASE']='models'
from p8fs.workers.dreaming import DreamingWorker
dreaming_worker = DreamingWorker()
user_data = await dreaming_worker.collect_user_data('tenant-zwwon1x8kni')
if user_data:
    print(len(user_data.resources))
else:
    print('no data')

2025-10-10 21:25:32.366 | DEBUG    | p8fs.repository.SystemRepository:__init__:39 - Initialized SystemRepository for Session without tenant scoping
2025-10-10 21:25:32.367 | DEBUG    | p8fs.repository.SystemRepository:__init__:39 - Initialized SystemRepository for Resources without tenant scoping
2025-10-10 21:25:32.367 | DEBUG    | p8fs.repository.SystemRepository:__init__:39 - Initialized SystemRepository for Tenant without tenant scoping
2025-10-10 21:25:32.368 | INFO     | p8fs.services.llm.base_proxy:__init__:109 - Initialized BaseProxy
2025-10-10 21:25:32.368 | INFO     | p8fs.services.llm.function_handler:__init__:167 - Initialized FunctionHandler
2025-10-10 21:25:32.368 | DEBUG    | p8fs.services.llm.function_handler:add_function:350 - Added function get_entities
2025-10-10 21:25:32.369 | DEBUG    | p8fs.services.llm.function_handler:add_function:350 - Added function search_resources
2025-10-10 21:25:32.369 | DEBUG    | p8fs.services.llm.function_handler:add_function:350 - Adde

-----------------------------------------------------------
-----------------------------------------------------------
-----------------------------------------------------------
0


In [2]:
from p8fs.models import AbstractModel
from p8fs.services.llm.memory_proxy import MemoryProxy
from p8fs.services.llm.models import CallingContext, BatchCallingContext
from p8fs import settings
tenant_id = settings.default_tenant_id
tenant_id

'tenant-test'

In [3]:
#subclass abstract model
class WeatherAgent(AbstractModel):
    """You are an agent that provides some weather services along with a joke or a poem as your party piece"""
    @classmethod
    def get_weather(self, location: str, units: str = "fahrenheit"):
        """Get current weather for a location."""
        temp = 72 if units == "fahrenheit" else 22
        return {
            "location": location,
            "temperature": temp,
            "units": units,
            "conditions": "sunny"
        }
    @classmethod
    def get_forecast(self, location: str, days: int = 3):
        """Get weather forecast for a location."""
        return {
            "location": location,
            "days": days,
            "forecast": [
                {"day": i+1, "high": 75+i, "low": 60+i, "conditions": "sunny"}
                for i in range(days)
            ]
        }

# Create agent and wrap with MemoryProxy
agent = MemoryProxy(WeatherAgent)

2025-10-10 21:25:32.569 | INFO     | p8fs.services.llm.base_proxy:__init__:109 - Initialized BaseProxy
2025-10-10 21:25:32.570 | INFO     | p8fs.services.llm.function_handler:__init__:167 - Initialized FunctionHandler
2025-10-10 21:25:32.571 | DEBUG    | p8fs.services.llm.function_handler:add_function:350 - Added function get_entities
2025-10-10 21:25:32.571 | DEBUG    | p8fs.services.llm.function_handler:add_function:350 - Added function search_resources
2025-10-10 21:25:32.572 | DEBUG    | p8fs.services.llm.function_handler:add_function:350 - Added function get_recent_tenant_uploads
2025-10-10 21:25:32.573 | DEBUG    | p8fs.services.llm.function_handler:add_function:350 - Added function get_forecast
2025-10-10 21:25:32.573 | DEBUG    | p8fs.services.llm.memory_proxy:_register_model_functions:271 - Registered function: get_forecast
2025-10-10 21:25:32.574 | DEBUG    | p8fs.services.llm.function_handler:add_function:350 - Added function get_weather
2025-10-10 21:25:32.574 | DEBUG    | 

In [4]:
F = agent._function_handler._functions['get_weather']
F('paris')

{'location': 'paris',
 'temperature': 72,
 'units': 'fahrenheit',
 'conditions': 'sunny'}

In [5]:
# Normal Mode - Complete Response
context = CallingContext(
    model="gpt-4.1" ,tenant_id = tenant_id
)

response = await agent.run("What's the weather in Chicago?", context)
response

2025-10-10 21:25:32.583 | DEBUG    | p8fs.services.llm.memory_proxy:stream:557 - Streaming agentic loop iteration 1/10
2025-10-10 21:25:33.712 | INFO     | p8fs.services.llm.function_handler:execute:244 - Executed function 'get_weather' successfully
2025-10-10 21:25:33.713 | DEBUG    | p8fs.services.llm.memory_proxy:_execute_function_calls:483 - Function get_weather executed successfully in streaming mode
2025-10-10 21:25:33.714 | DEBUG    | p8fs.services.llm.memory_proxy:stream:557 - Streaming agentic loop iteration 2/10
2025-10-10 21:25:36.944 | DEBUG    | p8fs.providers.postgresql:execute:294 - Executing query: INSERT INTO sessions (id, created_at, updated_at, name, query, user_rating, agent, parent_session_id...
2025-10-10 21:25:36.964 | DEBUG    | p8fs.repository.BaseRepository:upsert:404 - Successfully async upserted 1 Session entities


-----------------------------------------------------------


2025-10-10 21:25:37.559 | DEBUG    | p8fs.providers.postgresql:execute:294 - Executing query: 
            INSERT INTO embeddings.sessions_embeddings 
            (id, entity_id, field_name, emb...
2025-10-10 21:25:37.606 | INFO     | p8fs.repository.BaseRepository:_generate_embeddings_for_entities:860 - Generated and stored 1 embeddings in batch
2025-10-10 21:25:37.607 | DEBUG    | p8fs.services.llm.audit_mixin:start_audit_session:67 - Started audit session c4f59dc6-c493-4f26-be75-e50dd98413ad
2025-10-10 21:25:37.608 | DEBUG    | p8fs.services.llm.audit_mixin:track_usage:134 - Session c4f59dc6-c493-4f26-be75-e50dd98413ad: +5p +32c ($0.0021) Total: 37 tokens, $0.0021
2025-10-10 21:25:37.609 | INFO     | p8fs.services.llm.audit_mixin:end_audit_session:180 - Ended audit session c4f59dc6-c493-4f26-be75-e50dd98413ad: 37 tokens, 0 function calls, $0.0021, 0.70s
2025-10-10 21:25:37.647 | DEBUG    | p8fs.providers.postgresql:execute:294 - Executing query: INSERT INTO sessions (id, created_at,

-----------------------------------------------------------
-----------------------------------------------------------
-----------------------------------------------------------


2025-10-10 21:25:37.899 | DEBUG    | p8fs.providers.postgresql:execute:294 - Executing query: 
            INSERT INTO embeddings.sessions_embeddings 
            (id, entity_id, field_name, emb...
2025-10-10 21:25:37.917 | INFO     | p8fs.repository.BaseRepository:_generate_embeddings_for_entities:860 - Generated and stored 1 embeddings in batch


-----------------------------------------------------------


"The current weather in Chicago is sunny with a temperature of 72°F.\n\nAnd here's a quick weather-themed joke for you:\nWhy did the sun go to school? To get a little brighter!"

2025-10-10 21:25:38.432 | DEBUG    | p8fs.providers.postgresql:execute:294 - Executing query: 
            INSERT INTO embeddings.sessions_embeddings 
            (id, entity_id, field_name, emb...
2025-10-10 21:25:38.448 | INFO     | p8fs.repository.BaseRepository:_generate_embeddings_for_entities:860 - Generated and stored 1 embeddings in batch


-----------------------------------------------------------


In [ ]:
# Streaming Mode - Real-time Response
stream_context = CallingContext(stream=True)

async for chunk in agent.stream("Tell me about Miami weather", stream_context):
    if "choices" in chunk:
        content = chunk["choices"][0].get("delta", {}).get("content", "")
        print(content, end="", flush=True)


In [ ]:
# Batch Mode - Multiple Questions
questions = [
    "Weather in Boston?",
    "3-day forecast for Seattle?",
    "Temperature in Phoenix in Celsius?"
]

batch_context = BatchCallingContext(
    model="gpt-5",
    system_message="You are a weather assistant."
)

result = await agent.batch(questions, batch_context)
result

## Summary

You just learned the P8FS pattern:

```python
# 1. Create your agent class subclassing pydantic base model or our abstract types
class YourAgent(AbstractEntityModel):
    async def your_function(self, param: str):
        return {"result": "data"}

# 2. Wrap with MemoryProxy
agent = YourAgent
memory = MemoryProxy(agent)

# 3. Use in three modes
await memory.run(question, context)              # Normal
async for chunk in memory.stream(q, context):   # Streaming  
await memory.batch(questions, batch_context)    # Batch
```

**Next:** Add your LLM API key for real AI responses!

# Sessions and Files
- when you use memory proxy directly or via the completions API, sessions are audited which feed into dreaming.
- You can also process files locally to test simulating what happens when files are uploaded and processed in Seaweed; we index the file in Files table and we create chunks in Resources 

In [ ]:
from p8fs.repository.TenantRepository import TenantRepository
from p8fs.repository.SystemRepository import SystemRepository
from p8fs.models.p8 import Session, SessionType
from p8fs.providers import get_provider

# Create SystemRepository (bypasses tenant isolation)
repo = SystemRepository(Session)

recent_sessions = repo.execute("""
  SELECT * FROM sessions
  ORDER BY created_at DESC
  LIMIT 10
""")

recent_sessions

## Use the CLI to process files in the ./Samples folder or any file you choose
- the cli allows processing entire folders too

In [ ]:
! uv run -m p8fs.cli process ./p8fs/tests/sample_data/content/Sample.md 

In [ ]:
! uv run -m p8fs.cli process ./p8fs/tests/sample_data/content/Sample.pdf

In [ ]:
! uv run -m p8fs.cli process ./p8fs/tests/sample_data/content/Sample.wav

In [ ]:
! uv run -m p8fs.cli process ./p8fs/tests/sample_data/content/Sample.docx

# Now to Dreaming
- if we have sessions and resources we can use this as content to terst the dreaming agent
- normally this is managed by a process the dreaming worker and you can use the cli to run this as the docs show
- below we will use the dreaming worker code to get the data bundle and then simple use the Dreaming worker to process the data
- From here you can build your agents or play with the prompt

# Complete Dreaming Example - All Modes Working

This example demonstrates the full dreaming functionality with MemoryProxy and DreamingAgent.

In [16]:
# Complete Dreaming Example - Simple Version
import json
from datetime import datetime, timezone
from uuid import uuid4
from p8fs.models.agentlets import DreamModel
from p8fs.services.llm import MemoryProxy, CallingContext, BatchCallingContext

In [ ]:
from p8fs.workers.dreaming import DreamingWorker

repo = TenantRepository(Session, tenant_id)
#dreaming worker just uses to fetch user data as we do in the worker process e.g. 24 hours of data
dreaming_worker = DreamingWorker()
agent = MemoryProxy(DreamModel)

# Collect real user data using dreaming worker utility as the scheduled job would do
# in this case our own activity can create these records:> files processed and chats had.
user_data = await dreaming_worker.collect_user_data(tenant_id)

In [ ]:
#add any args required
from IPython.display import Markdown
context = CallingContext(tenant_id=tenant_id,)

#in practice dreaming worker should handle large user data batches
query = f"""Analyze this user activity data: {user_data.model_dump()} Provide key insights and recommendations."""

response = await agent.run(query, context)
Markdown(response)

In [ ]:
# async for chunk in memory_proxy.stream(query, context):
#     chunks += 1
#     #TODO print friendly sse
#     if chunks > 5: break

## Mostly we do dreaming in batch mode

In [ ]:
batch_context = BatchCallingContext(
    model="gpt-5", 
    tenant_id=tenant_id
)

batch_response = await agent.batch(query, batch_context)
batch_response